# Lecture 3: Autodiff in JAX

In [ ]:
import jax
import jax.numpy as np
from jax import grad, vmap

import matplotlib.pyplot as plt

In [ ]:
jax.print_environment_info()

In [ ]:
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

In [ ]:
sigmoid(0.0)

In [ ]:
xs = np.arange(-5, 5, 0.1)
plt.plot(xs, sigmoid(xs))
plt.show()

In [ ]:
from jax import make_jaxpr
make_jaxpr(sigmoid)(1.0)

In [ ]:
f = sigmoid
# f = np.tanh
fp = grad(f)

In [ ]:
fp(0.0)

In [ ]:
v_fp = vmap(fp)

xs = np.arange(-5, 5, 0.1)
plt.plot(xs, f(xs))
plt.plot(xs, v_fp(xs))
plt.show()

In [ ]:
make_jaxpr(fp)(1.0)

In [ ]:
fpp = grad(fp)
fppp = grad(fpp)

In [ ]:
fpp(0.0)

In [ ]:
fppp(0.0)

In [ ]:
v_fpp = vmap(fpp)
v_fppp = vmap(fppp)

xs = np.arange(-5, 5, 0.1)
plt.plot(xs, f(xs), label="sigmoid(x)")
plt.plot(xs, v_fp(xs), label="sigmoid'(x)")
plt.plot(xs, v_fpp(xs), label="sigmoid''(x)")
plt.plot(xs, v_fppp(xs), label="sigmoid'''(x)")
plt.legend()
plt.show()

In [ ]:
make_jaxpr(fpp)(0.0)

In [ ]:
make_jaxpr(fppp)(0.0)